In [1]:
import Pkg;
Pkg.add("ADTypes")
Pkg.add("Optim")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed ADTypes ─ v1.21.0
    Updating `~/.julia/environments/v1.12/Project.toml`
  [47edcb42] + ADTypes v1.21.0
    Updating `~/.julia/environments/v1.12/Manifest.toml`
  [47edcb42] + ADTypes v1.21.0
Precompiling packages...
   1806.2 ms  ✓ ADTypes
   1711.2 ms  ✓ ADTypes → ADTypesChainRulesCoreExt
  2 dependencies successfully precompiled in 6 seconds. 313 already precompiled.
   Resolving package versions...
   Installed EnumX ──────────────────── v1.0.6
   Installed PositiveFactorizations ─── v0.2.4
   Installed NLSolversBase ──────────── v8.0.0
   Installed ConstructionBase ───────── v1.6.0
   Installed Setfield ───────────────── v1.1.2
   Installed LineSearches ───────────── v7.6.0
   Installed FiniteDiff ─────────────── v2.29.0
   Installed ArrayInterface ─────────── v7.22.0
   Installed Optim ──────────────────── v2.0.1
   Installed DifferentiationInterface ─ v0.7.16
    Updating `

In [2]:
using ADTypes, LinearAlgebra, Optim, Plots, Printf, ForwardDiff

## P1. Conditioning Experiment on a Quadratic

Recall from last worksheet our Julia implementation of gradient method with exact linesearch for minimizing quadratic functions of the form

$$
f(x) = \frac12 x^T Ax + b^T x.
$$

(The function below is slightly modified from last time to make your first experiment easier.)

**Your first task**: modify the function so that it takes an additional parameter $M$ (which you can assume without checking is positive definite) and sets the descent step at timestep $k$ to be equal to $-M^{-1}\nabla f$. Note the *inverse* in this version (but remember that you shouldn't use inv()!).

In [21]:
function descent_method_exact_ls(A, b, x0, ϵ=1e-6,)
    x = copy(x0)
    ∇f = A*x + b
    k = 0
    xtrace = x'
    while norm(∇f) > ϵ
        d = -∇f
        α = -dot(∇f,d) / dot(d,A*d)
        x = x + α*d
        ∇f = A*x + b
        f = (1/2)x'*A*x + b'*x
        @printf "it = %3d | |∇f| = %8.2e | f = %8.2e\n" k norm(∇f) f
        k += 1
        xtrace = vcat(xtrace,x')
    end
    return xtrace
end

descent_method_exact_ls (generic function with 3 methods)

In [22]:
function descent_method_scaled(A, b, x0, M, ϵ=1e-6,)
    x = copy(x0)
    ∇f = A*x + b
    k = 0
    xtrace = x'
    while norm(∇f) > ϵ
        d = -inv(M)*∇f # <- should b ed = -(M\ grad-f) 
        α = -dot(∇f,d) / dot(d,A*d)
        x = x + α*d
        ∇f = A*x + b
        f = (1/2)x'*A*x + b'*x
        @printf "it = %3d | |∇f| = %8.2e | f = %8.2e\n" k norm(∇f) f
        k += 1
        xtrace = vcat(xtrace,x')
    end
    return xtrace
end

descent_method_scaled (generic function with 2 methods)

Next, consider optimizing the function 
$$
q(x) = \frac12 x^T A x,
$$
where
$$
A = \begin{bmatrix}
1 & -0.1 \\
-0.1 & 10
\end{bmatrix},
$$
Compare the convergence of (i) gradient descent, (ii) scaled gradient descent with $M = \text{diag}(A) = \text{diag}(1, 10)$, and (iii) scaled gradient descent with $M = A$, all beginning from the point $(1,2)$.

In [ ]:
## your experiments go here. Make sure you add your commentary to the worksheet!

A = [1. -0.1; -0.1 10]
b = [0; 0] 

descent_method_exact_ls(A, b, [1, 2])

it =   0 | |∇f| = 9.20e-01 | f = 4.23e-01
it =   1 | |∇f| = 4.15e-01 | f = 8.80e-03
it =   2 | |∇f| = 1.92e-02 | f = 1.83e-04
it =   3 | |∇f| = 8.63e-03 | f = 3.82e-06
it =   4 | |∇f| = 3.99e-04 | f = 7.94e-08
it =   5 | |∇f| = 1.80e-04 | f = 1.65e-09
it =   6 | |∇f| = 8.30e-06 | f = 3.44e-11
it =   7 | |∇f| = 3.74e-06 | f = 7.17e-13
it =   8 | |∇f| = 1.73e-07 | f = 1.49e-14
descent_method_exact_ls(A, b, [1, 2]) = [1.0 2.0; 0.9198192015476219 0.005502638497092649; 0.02082162234803553 0.04164324469607102; 0.01915212804309616 0.00011457386070423681; 0.00043353995720411126 0.000867079914408324; 0.00039877837727447676 2.3856136585392786e-6; 9.027005261687238e-6 1.8054010523374747e-5; 8.303212772171338e-6 4.967234666642179e-8; 1.8795689449254908e-7 3.7591378898513125e-7; 1.7288636061750744e-7 1.03425884342876e-9]


10×2 Matrix{Float64}:
 1.0          2.0
 0.919819     0.00550264
 0.0208216    0.0416432
 0.0191521    0.000114574
 0.00043354   0.00086708
 0.000398778  2.38561e-6
 9.02701e-6   1.8054e-5
 8.30321e-6   4.96723e-8
 1.87957e-7   3.75914e-7
 1.72886e-7   1.03426e-9

In [27]:
descent_method_scaled(A, b, [1, 2], diagm([1, 10]))

it =   0 | |∇f| = 2.09e-01 | f = 1.90e-02
it =   1 | |∇f| = 1.87e-02 | f = 1.78e-05
it =   2 | |∇f| = 1.96e-04 | f = 1.67e-08
it =   3 | |∇f| = 1.75e-05 | f = 1.57e-11
it =   4 | |∇f| = 1.84e-07 | f = 1.47e-14


6×2 Matrix{Float64}:
 1.0           2.0
 0.19362      -0.00587111
 0.000937454   0.00187491
 0.000181509  -5.50389e-6
 8.7882e-7     1.75764e-6
 1.70157e-7   -5.15965e-9

In [28]:
descent_method_scaled(A, b, [1, 2], A)

it =   0 | |∇f| = 4.46e-16 | f = 9.86e-32


2×2 Matrix{Float64}:
  1.0          2.0
 -4.44089e-16  0.0

### P2. Using different scaled descent solvers


In [ ]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x0 = [0.0, 0.0]
opts = Optim.Options(store_trace=true, extended_trace=true);

In [ ]:
res = []
for method in [GradientDescent, BFGS, Newton]
   r = optimize(f, x0, method(), autodiff = ADTypes.AutoForwardDiff(), opts)
   push!(res, r)
end

trace = res[1].trace
fvals = [t.value for t in trace]
plot(fvals[1:100], label="gradient descent")
trace = res[2].trace
fvals = [t.value for t in trace]
plot!(fvals, label="BFGS (quasi Newton)")
trace = res[3].trace
fvals = [t.value for t in trace]
plot!(fvals, label="Newton")